In [9]:
import pymongo
from pymongo import MongoClient
import pprint
from IPython.display import clear_output
import connect as c
# Replace XXXX with your connection URI from the Atlas UI
client = c.getMongoClient()

# Like the last handout, this pipeline will not work on Atlas until MongoDB 3.6 has been released
# If you're testing this before 3.6 is released you can download and install MongoDB 3.5.X locally
# In that case you should use "mongodb://localhost:27017" as your connection URI
pipeline = [
    {
        '$limit': 100
    },
    {
        '$addFields': {
            'lastupdated': {
                '$arrayElemAt': [
                    {'$split': ["$lastupdated", "."]},
                    0
                ]}
        }
    },
    {
        '$project': {
            'title': 1,
            'year': 1,
            'directors': {'$split': ["$director", ", "]},
            'actors': {'$split': ["$cast", ", "]},
            'writers': {'$split': ["$writer", ", "]},
            'genres': {'$split': ["$genre", ", "]},
            'languages': {'$split': ["$language", ", "]},
            'countries': {'$split': ["$country", ", "]},
            'plot': 1,
            'fullPlot': "$fullplot",
            'rated': "$rating",
            'released': {
                '$cond': {
                    'if': {'$ne': ["$released", ""]},
                    'then': {
                        '$dateFromString': {
                            'dateString': "$released"
                        }
                    },
                    'else': ""}},
            'runtime': 1,
            'poster': 1,
            'imdb': {
                'id': "$imdbID",
                'rating': "$imdbRating",
                'votes': "$imdbVotes"
                },
            'metacritic': 1,
            'awards': 1,
            'type': 1,
            'lastUpdated': {
                '$cond': {
                    'if': {'$ne': ["$lastupdated", ""]},
                    'then': {
                        '$dateFromString': {
                            'dateString': "$lastupdated",
                            'format': "%H:%M",
                            'timezone': "America/New_York",
                            'onNull': "The date was either empty or null"
                        }
                    },
                    'else': ""}}
        }
    },
    {
        '$out': "movies_scratch"
    }
]
clear_output()
pprint.pprint(list(client.mflix.movies_initial.aggregate(pipeline)))

OperationFailure: an incomplete date/time string has been found, with elements missing: "03:45", full error: {'operationTime': Timestamp(1632681108, 1), 'ok': 0.0, 'errmsg': 'an incomplete date/time string has been found, with elements missing: "03:45"', 'code': 241, 'codeName': 'ConversionFailure', '$clusterTime': {'clusterTime': Timestamp(1632681108, 2), 'signature': {'hash': b'\x00\xb0\x92\xbb\xe3 \xea\x872A\xa3g\xe4q\x10\x8a\xfd q\xa8', 'keyId': 6952520521338585089}}}